In [2]:
%pip install pandas
%pip install gradio
%pip install folium
%pip install geopy

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import gradio as gr
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic

#데이터 로드

# 데이터프레임 로드
APT = pd.read_csv('./export/MaintrainDatas.csv')
APT = APT.dropna(subset=['lat', 'lon'])

# 교육시설 데이터 로드
EDU = pd.read_csv('./export/education/final_graded_school_locations.csv')
EDU = EDU[['cluster'	,'DistanceToCenter'	,'등급','학교명','조사년도','개방ID','lat','lon','score']]
EDU = EDU.rename(columns={'등급': 'class'})
EDU = EDU.dropna(subset=['lat','lon'])

Medicals = pd.read_csv('./export/hospital/hospitalOutput.csv')
Medicals = Medicals[['암호화요양기호','요양기관명','cluster','DistanceToCenter','score','class','lat','lon']]
Medicals =  Medicals.dropna(subset=['lat','lon','score'])

policeFacilities = pd.read_csv('./export/safety/policeOutput.csv')
policeFacilities =  policeFacilities.dropna(subset=['lat','lon'])
policeFacilities = policeFacilities.rename(columns={'구분': 'class'})
policeFacilities['score'] = policeFacilities['class'] 

fireCentor = pd.read_csv('./export/safety/filtered_safety_center_data_seoul.csv')
fireCentor = fireCentor.dropna(subset=['lat','lon'])
fireCentor = fireCentor.rename(columns={'구분': 'class'})
fireCentor['score'] = fireCentor['class'] 

# 서울시의 구 목록
districts = APT['주소(시군구)'].unique().tolist()


guPos = pd.read_csv('./data/seoul.csv')
guPos = guPos.rename(columns={'District Name': 'name','Latitude':'lat','Longitude':'lon'})



c:\Users\fenpon\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#UI 작동

lookScore = ['score','safetyScore']
lookClass = ['class','safetyClass']
isApart = False
# 선택한 구의 아파트 목록을 가져오는 함수
def get_apartments(selected_district, lookType):
    
    isApart = False
    filtered_apartments = APT[APT['주소(시군구)'] == selected_district]

    myGu = guPos[guPos['name'] == selected_district]
    if myGu.empty:
        return "해당 구의 위치 정보가 없습니다."
    
    guLocation = [myGu['lat'], myGu['lon']]
    
    m = folium.Map(location=guLocation, zoom_start=13)
  
    apartment_names = filtered_apartments['k-아파트명'].unique().tolist()
    dropDownIndex = 0
    for i, row in filtered_apartments.iterrows():
        apartName = row['k-아파트명']
        # JavaScript 코드와 버튼 HTML
     
        print(i)
        # 구의 아파트 선택시 이벤트 연결 부분인데 작동을 안함
        
        popup_html = f"""
            <button onclick="(function() {{
                    console.log('yes');
                    var dropdown = document.getElementById('apartDrop');
                    console.log(dropdown);
                    dropdown.selectedIndex = `{dropDownIndex}`;
            }})();">
                {row['k-아파트명']} {row['score']}
            </button>
        """
                    #var event = new Event('change');
                    #dropdown.dispatchEvent(event);
        # 위치 및 마커 설정
        apartment_location = [row['lat'], row['lon']]
        colors = ['yellow', 'green', 'blue']
        cc = colors[int(row[lookClass[lookType]])]
        dropDownIndex += 1

        folium.Marker(
            location=apartment_location,
            popup=folium.Popup(f"{row['k-아파트명']} ({row[lookScore[lookType]]})", max_width=250),
            icon=folium.Icon(color=cc, icon_color=cc)
        ).add_to(m)

    return (gr.Dropdown(choices=apartment_names , label="아파트 선택", interactive=True), m._repr_html_())
 # set_choices 사용

# 아파트 주변 1km 이내 교육시설 필터링 및 추가
def CreateMarker(m,apartment_location,nameTag,df,fileName = None):
    nearby_Cols = []

    for _, row in df.iterrows():
        locations = [row['lat'], row['lon']]
        distance = geodesic(apartment_location, locations).km
        if distance < 3:
            nearby_Cols.append({
                "name": row[nameTag],
                "location": locations,
                "distance": distance,
                'class' : int(row['class']),
                'score' : row['score']
            })

    

    for now in nearby_Cols:
        myIcon = None
        if fileName is None:
            colors = ['yellow','green','blue']
            myIcon = folium.Icon(icon="circle", color='gray', icon_size=(20, 20))
        else :
            icon_path = f'./icon/{fileName}{str(now["class"])}.png'  # Correct string interpolation
            myIcon = folium.CustomIcon(icon_path, icon_size=(35, 30)) 
  
        
        # Popup에 JavaScript와 HTML 버튼 추가
        popup_html = f"""
            <b>{now['name']} ({now['distance']:.2f} km) ({now['score']})</b><br>
        """
        
        folium.Marker(
            location=now['location'],
            popup=popup_html ,
            icon= myIcon
        ).add_to(m)
# 선택된 아파트 주변의 교육시설과 아파트 위치를 지도에 표시하는 함수
def OnChange(selected_district, selected_apartment,m):
    print('Now Loading')
    isApart = True
    # 선택된 아파트 데이터 필터링
    filtered_apartments = APT[(APT['주소(시군구)'] == selected_district) & (APT['k-아파트명'] == selected_apartment)]
    if filtered_apartments.empty:
        return "아파트 정보를 찾을 수 없습니다."

    apartment_row = filtered_apartments.iloc[0]
    apartment_location = [apartment_row['lat'], apartment_row['lon']]

    # 지도 초기화 (아파트 위치로 중심 설정)
    m = folium.Map(location=apartment_location, zoom_start=15)
   # m.zoom_start = 15  # Optionally change the zoom level

    marker_cluster = MarkerCluster().add_to(m)

    
    # 아파트 마커 추가
    folium.Marker(
        location=apartment_location,
        popup=f"{selected_apartment} (아파트) ",
        icon=folium.Icon(color='red')
    ).add_to(marker_cluster)
    #CreateEdu(m,apartment_location)
    CreateMarker(m,apartment_location,'학교명',EDU,'study')
    CreateMarker(m,apartment_location,'요양기관명',Medicals,'medical')
    CreateMarker(m,apartment_location,'k-아파트명',APT,'home')
    CreateMarker(m,apartment_location,'이름',policeFacilities,'police')
    #소방 아이콘 추가하기
    
    return m._repr_html_()


# Gradio 인터페이스 구성
with gr.Blocks() as iface:
    gr.Markdown("## 서울시 아파트 지도")
    map_output = None
    with gr.Row():
        with gr.Column(scale=1):
            with gr.Column():
                with gr.Row():

                # 구 선택 드롭다운
                    district_dropdown = gr.Dropdown(choices=districts, label="구 선택")
                    print(district_dropdown)
                    apartment_dropdown = gr.Dropdown(choices=[],elem_id='apartDrop', label="아파트 선택", interactive=True)
                    
                with gr.Row():
                        gr.Markdown('<div style="font-size: 32px; font-family: Arial; ">설정 : </div>')
                        totalRoi = gr.Button("종합 점수")
                        safetyRoi = gr.Button("치안 보기")
                    
                       
            with gr.Column():
                # 지도 출력
                map_output = gr.HTML(label="서울시 아파트 지도")
               
                # 구 선택 시 아파트 목록 업데이트
                district_dropdown.change(
                    fn=get_apartments,
                    inputs=[district_dropdown, gr.State(0)],  # lookType=0 (기본 값, 종합 점수)
                    outputs=(apartment_dropdown, map_output)
                )

              
                # 버튼 클릭 시 점수 유형에 맞춰 아파트 목록 업데이트
      
                # 종합 점수 버튼 클릭 시
                totalRoi.click(
                    fn=get_apartments,
                    inputs=[district_dropdown, gr.State(0)],  # 구 선택값과 lookType=0 (종합 점수)
                    outputs=[apartment_dropdown, map_output]
                )

                safetyRoi.click(
                    fn=get_apartments,
                    inputs=[district_dropdown, gr.State(1)],  # 구 선택값과 lookType=1 (치안 보기)
                    outputs=[apartment_dropdown, map_output]
                )
                # 아파트 선택 시 지도 업데이트
                apartment_dropdown.change(fn=OnChange, inputs=[district_dropdown, apartment_dropdown,map_output], outputs=map_output)
                live = True
                # 설명 추가
                gr.Markdown("구를 선택한 후 아파트를 선택하면 해당 아파트와 주변 교육시설이 지도에 표시됩니다.")

# Gradio 인터페이스 실행
iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://cee800847ec6279a06.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


c:\Users\fenpon\AppData\Local\Programs\Python\Python310\lib\site-packages\folium\utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
c:\Users\fenpon\AppData\Local\Programs\Python\Python310\lib\site-packages\folium\utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
c:\Users\fenpon\AppData\Local\Programs\Python\Python310\lib\site-packages\folium\utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]
C:\Users\fenpon\AppData\Local\Temp\ipykernel_29652\2996887802.py:50: UserWarning: color argument of Icon should be one of: {'blue', 'lightred', 'pink', 'lightgreen', 'green', 'cadetblue', 'red'

4
18
23
48
87
100
125
137
138
165
202
203
207
214
244
246
269
298
318
328
338
339
347
400
401
413
414
427
437
472
475
501
507
508
535
552
553
556
571
602
609
623
624
632
642
648
662
670
695
697
698
720
727
729
736
759
773
808
821
848
851
856
875
895
896
907
919
926
928
955
957
963
964
973
985
989
990
993
994
1007
1012
1020
1023
1033
1035
1040
1041
1047
1057
1064
1082
1085
1100
1132
1134
1139
1146
1155
1177
1184
1206
1211
1212
1218
1233
1241
1247
1254
1263
1267
1269
1272
1274
1275
1288
1297
1325
1336
1339
1347
1351
1376
1378
1381
1383
1386
1398
1403
1415
1427
1431
1436
1510
1528
1533
1538
1540
1555
1557
1580
1585
1598
1604
1609
1616
1617
1619
1620
1622
1628
1630
1631
1635
1637
1642
1647
1650
1651
1656
1662
1663
1665
1666
1670
1675
1681
1684
1689
1702
1717
1731
1737
1741
1755
1780
1782
1793
1795
1806
1809
1825
1828
1871
1872
1910
1911
1961
1974
1982
2025
2029
2097
2191
2251
2259
2260
2272
2273
2329
2347
2382
2397
2491
2498
2530
2542
2543
2564
2577
2644
2645
2703
2736
2738
4
18
23
48
87
1

In [5]:
pip install --upgrade gradio

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
